![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = 'UCI/features.txt'
filename_labels = 'UCI/activity_labels.txt'

filename_subtrain = 'UCI/train/subject_train.txt'
filename_xtrain = 'UCI/train/X_train.txt'
filename_ytrain = 'UCI/train/y_train.txt'

filename_subtest = "UCI/test/subject_test.txt"
ffilename_xtest = 'UCI/test/X_test.txt'
filename_ytest =  'UCI/test/y_test.txt'

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features['nome_var'].tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [8]:


clf = DecisionTreeClassifier(random_state=100, ccp_alpha=0.001)

In [12]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer

# Criar um imputer para preencher valores ausentes
imputer = SimpleImputer(strategy='mean')  # Estratégia de preenchimento pode variar

# Preencher valores ausentes nos conjuntos de treino e teste
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Inicializar e treinar o modelo HistGradientBoostingClassifier
clf = HistGradientBoostingClassifier()
clf.fit(X_train_imputed, y_train)

/home/carolina/.local/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(
/home/carolina/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


HistGradientBoostingClassifier()

## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [17]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

# Cria um imputer para preencher os valores ausentes (NaN) com a média dos dados
imputer = SimpleImputer(strategy='mean')

# Aplica a imputação nos dados de treinamento
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Aplica o PCA nos dados imputados
pca = PCA(n_components=1)  # Define o número de componentes desejado
pc_train = pca.fit_transform(X_train_imputed)
pc_test = pca.transform(X_test_imputed)


In [18]:
n = 1

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_train[:,:n], columns = colunas)
pc_test  = pd.DataFrame( pc_test[:,:n], columns = colunas)

pc_train.head()

,cp1
0,-5.520280
1,-5.535350
2,-5.474988
3,-5.677232
4,-5.748749


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [27]:
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA

print("Valores ausentes em X_train antes da imputação:")
print(X_train.isnull().sum())


imputer = SimpleImputer(strategy='mean')

X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


print("\nValores ausentes em X_train após a imputação:")
print(pd.DataFrame(X_train_imputed).isnull().sum())

pca = PCA(n_components=10)  
pc_train = pca.fit_transform(X_train_imputed)
pc_test = pca.transform(X_test_imputed)

print(f'\nDimensões dos dados de treinamento após o PCA: {pc_train.shape}')
print(f'Dimensões dos dados de teste após o PCA: {pc_test.shape}')


Valores ausentes em X_train antes da imputação:
subject,,"(1, 'tBodyAcc-mean()-X')","(2, 'tBodyAcc-mean()-Y')","(3, 'tBodyAcc-mean()-Z')","(4, 'tBodyAcc-std()-X')","(5, 'tBodyAcc-std()-Y')","(6, 'tBodyAcc-std()-Z')","(7, 'tBodyAcc-mad()-X')","(8, 'tBodyAcc-mad()-Y')","(9, 'tBodyAcc-mad()-Z')","(10, 'tBodyAcc-max()-X')","(11, 'tBodyAcc-max()-Y')","(12, 'tBodyAcc-max()-Z')","(13, 'tBodyAcc-min()-X')","(14, 'tBodyAcc-min()-Y')","(15, 'tBodyAcc-min()-Z')","(16, 'tBodyAcc-sma()')","(17, 'tBodyAcc-energy()-X')","(18, 'tBodyAcc-energy()-Y')","(19, 'tBodyAcc-energy()-Z')","(20, 'tBodyAcc-iqr()-X')","(21, 'tBodyAcc-iqr()-Y')","(22, 'tBodyAcc-iqr()-Z')","(23, 'tBodyAcc-entropy()-X')","(24, 'tBodyAcc-entropy()-Y')","(25, 'tBodyAcc-entropy()-Z')","(26, 'tBodyAcc-arCoeff()-X,1')","(27, 'tBodyAcc-arCoeff()-X,2')","(28, 'tBodyAcc-arCoeff()-X,3')","(29, 'tBodyAcc-arCoeff()-X,4')","(30, 'tBodyAcc-arCoeff()-Y,1')","(31, 'tBodyAcc-arCoeff()-Y,2')","(32, 'tBodyAcc-arCoeff()-Y,3')","(33, 'tBodyAcc-arCoeff(

## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

No inicio conforme aumentamos o número de componentes aumentamos consideravelmente a acurácia de teste se mantém praticamente constante e a da base treino continua aumentando. Percebemos que não precisamos utilizar muitos componentes para chegarmos em uma acurácia de testes boa, porém também não podemos usar quase nenhum componente, pois também prejudicariamos o modelo.